In [1]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.utils.sampling import get_polytope_samples

def constraint(X):
    return torch.sum(X[...,:2], dim=-1) -1.80

train_X = torch.rand(10, 3)
Y = 1 - torch.norm(train_X[:,:2] - 0.5, dim=-1, keepdim=True)
Y = Y + 0.1 * torch.randn_like(Y)  # add some noise
train_Y = standardize(Y)

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

from botorch.optim import optimize_acqf


bounds = torch.stack([torch.zeros(3), torch.ones(3)])

batch_initial_conditions = get_polytope_samples(
    n=5,
    bounds=bounds,
    inequality_constraints=[(torch.tensor([0,1]),torch.tensor([1.,1.]),1.8)],
    n_burnin=1000
).unsqueeze(-2)

#batch_initial_conditions[...,-1] = 0.51


candidate, acq_value = optimize_acqf(
    UCB, 
    bounds=bounds, 
    q=1, 
    num_restarts=5, 
    raw_samples=20,
    fixed_features={2:0.5}, 
    nonlinear_inequality_constraints = [constraint],
    batch_initial_conditions = batch_initial_conditions
)
print(candidate)  # tensor([0.4887, 0.5063])
print(candidate.sum())

/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[1.0000, 0.8000, 0.5000]])
tensor(2.3000)


/Users/j30607/sandbox/botorch/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)


In [18]:
#batch_initial_conditions[...,-1] = 0.51

batch_initial_conditions

tensor(1.7622)

In [23]:
0.2409+0.3020-0.5

0.04289999999999994

In [26]:
test

tensor([[[0.2409, 0.3020, 0.8284],
         [0.9964, 0.2340, 0.2588]],

        [[0.8168, 0.1836, 0.4096],
         [0.9400, 0.1572, 0.7149]],

        [[0.8562, 0.6386, 0.2433],
         [0.0539, 0.6275, 0.2009]],

        [[0.3394, 0.0418, 0.9832],
         [0.0866, 0.8408, 0.6403]],

        [[0.5344, 0.4981, 0.0286],
         [0.3925, 0.0441, 0.2758]],

        [[0.4395, 0.7974, 0.1505],
         [0.0244, 0.4612, 0.5825]],

        [[0.3437, 0.0486, 0.8325],
         [0.1474, 0.1788, 0.0014]],

        [[0.9382, 0.4493, 0.9168],
         [0.3761, 0.5967, 0.8065]]])

In [27]:
torch.sum(test[...,:2], dim=-1)-0.5

tensor([[ 0.0429,  0.7304],
        [ 0.5004,  0.5972],
        [ 0.9947,  0.1813],
        [-0.1188,  0.4273],
        [ 0.5325, -0.0634],
        [ 0.7369, -0.0144],
        [-0.1077, -0.1738],
        [ 0.8875,  0.4728]])

In [11]:
from bofire.data_models.surrogates.api import TanimotoGPSurrogate, AnySurrogate
from bofire.data_models.features.api import MolecularInput, ContinuousOutput
from bofire.data_models.domain.api import Inputs, Outputs
from bofire.data_models.molfeatures.api import Fragments

import pandas as pd

inputs = Inputs(features=[MolecularInput(key="x_1")])
outputs = Outputs(features=[ContinuousOutput(key="y")])
experiments = [
    ["CC(=O)Oc1ccccc1C(=O)O", 88.0],
    ["c1ccccc1", 35.0],
    ["[CH3][CH2][OH]", 69.0],
    ["N[C@](C)(F)C(=O)O", 20.0],
]
experiments = pd.DataFrame(experiments, columns=["x_1", "y"])

surrogate_data = TanimotoGPSurrogate(
        inputs=inputs,
        outputs=outputs,
        input_preprocessing_specs={"x_1": Fragments()},
    )


# "kernel, specs",
#     [
#         (
#             ScaleKernel(base_kernel=TanimotoKernel(ard=False)),
#             {"x_1": Fingerprints(n_bits=32)},
#         ),
#         (
#             ScaleKernel(base_kernel=TanimotoKernel(ard=True)),
#             {"x_1": Fragments()},
#         ),
#         (
#             ScaleKernel(base_kernel=TanimotoKernel(ard=False)),
#             {"x_1": FingerprintsFragments(n_bits=32)},
#         ),
#         (
#             ScaleKernel(base_kernel=RBFKernel(ard=True)),
#             {"x_1": MordredDescriptors(descriptors=["NssCH2", "ATSC2d"])},
#         ),
#     ],
# )
# def test_TanimotoGP(kernel, specs):
#     inputs = Inputs(features=[MolecularInput(key="x_1")])
#     outputs = Outputs(features=[ContinuousOutput(key="y")])
#     experiments = [
#         ["CC(=O)Oc1ccccc1C(=O)O", 88.0],
#         ["c1ccccc1", 35.0],
#         ["[CH3][CH2][OH]", 69.0],
#         ["N[C@](C)(F)C(=O)O", 20.0],
#     ]
#     experiments = pd.DataFrame(experiments, columns=["x_1", "y"])
#     experiments["valid_y"] = 1
#     model = TanimotoGPSurrogate(
#         inputs=inputs,
#         outputs=outputs,
#         kernel=kernel,
#         input_preprocessing_specs=specs,
#     )

In [3]:
surrogate_data.input_preprocessing_specs

{'x_1': Fingerprints(type='Fingerprints', bond_radius=5, n_bits=2048)}

In [7]:
dump = surrogate_data.dict()

In [5]:
from pydantic import parse_obj_as

In [12]:
dump = surrogate_data.dict()
parse_obj_as(AnySurrogate,dump).input_preprocessing_specs

{'x_1': Fragments(type='Fragments', fragments=None)}